In [1]:
pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import tensorflow as tf
import sklearn
import scikeras
import time
from scikeras.wrappers import KerasRegressor
from tensorflow.keras import backend as k 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn import metrics

In [3]:
inicio = time.time()

In [3]:
base = pd.read_csv('autos.csv', encoding = 'ISO-8859-1')

In [4]:
base = base.drop('dateCrawled', axis = 1)
base = base.drop('dateCreated', axis = 1)
base = base.drop('nrOfPictures', axis = 1)
base = base.drop('postalCode', axis = 1)
base = base.drop('lastSeen', axis = 1)
base = base.drop('name', axis = 1)
base = base.drop('seller', axis = 1)
base = base.drop('offerType', axis = 1)

In [5]:
base = base[base.price > 10]
base = base.loc[base.price < 350000]

In [6]:
valores = {'vehicleType': 'limousine', 'gearbox': 'manuell', 'model': 'golf', 'fuelType': 'benzin', 'notRepairedDamage': 'nein'}
base = base.fillna(value = valores)

In [7]:
X = base.iloc[:, 1:12].values
y = base.iloc[:, 0].values

In [8]:
onehotencoder = ColumnTransformer(transformers = [("OneHot", OneHotEncoder(), [0, 1, 3, 5, 8, 9, 10])], remainder = 'passthrough') #Transformando variáveis categóricas em variáveis binárias
X = onehotencoder.fit_transform(X).toarray()

In [9]:
def criar_rede():
    k.clear_session()
    regressor = Sequential([
        tf.keras.layers.Input(shape = (316,)),
        tf.keras.layers.Dense(units = 158, activation = 'relu'),
        tf.keras.layers.Dense(units = 158, activation = 'relu'),
        tf.keras.layers.Dense(units = 1, activation = 'linear')
    ])
    regressor.compile(optimizer = 'adam', loss = 'mean_absolute_error', metrics = ['mean_absolute_error'])
    return regressor

In [10]:
regressor = KerasRegressor(model = criar_rede, epochs = 100, batch_size = 300)

In [11]:
resultados = cross_val_score(estimator = regressor, X = X, y = y, cv = 5, scoring = 'neg_mean_absolute_error')


Epoch 1/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 4441.6924 - mean_absolute_error: 4441.6924
Epoch 2/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 3504.0571 - mean_absolute_error: 3504.0571
Epoch 3/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 3268.1230 - mean_absolute_error: 3268.1230
Epoch 4/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 3077.1313 - mean_absolute_error: 3077.1313
Epoch 5/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 2959.8726 - mean_absolute_error: 2959.8726
Epoch 6/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 2858.6768 - mean_absolute_error: 2858.6768
Epoch 7/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 2768.4514 - mean_absolute_error: 2768.4514
Epoch 8/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 2739.9668 - mean_absolute_error: 2739.9668
Epoch 9/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 2727.5547 - mean_absolute_error: 2727.5544
Epoch 10/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 2

In [18]:
abs(resultados)

array([2316.96000761, 2251.87574   , 2395.86788864, 2231.95296581,
       2172.30529907])

In [19]:
abs(resultados.mean())

2273.792380224694

In [20]:
resultados.std()

76.54590409429464